In [ ]:
import numpy
import numpy as np
import pandas as pd
from glob import glob
import holoviews as hv
import funcs
hv.notebook_extension()
%opts Image {+axiswise} [colorbar=True aspect='square'] (cmap='viridis')

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/208px-3D_Spherical.svg.png)

# Varying $\mu$ (_has_ $\vec{A}$ correction for supercurrent in SC)

In [ ]:
funcs.combine_dfs("conductance_*_sc_correction.hdf")

In [ ]:
df = pd.read_hdf('data/conductance_sc_correction.hdf')
df['angle'] = 0
df['B'], df['theta'], df['phi'] = funcs.cartesian_coords(df.B_x, df.B_y, df.B_z)
params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'theta', 'phi']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
df2 = funcs.drop_constant_columns(df)
df2.columns

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)

# Varying $\mu$ (has no $\vec{A}$ correction for supercurrent in SC)

In [ ]:
df = pd.read_hdf('data/conductance_vary_mu.hdf')
df['B'], df['theta'], df['phi'] = funcs.cartesian_coords(df.B_x, df.B_y, df.B_z)

params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'theta', 'phi']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)

# First data, $\vec{B}$ in different directions


In [ ]:
df = pd.read_hdf('data/conductance.hdf')
df['B'], df['theta'], df['phi'] = funcs.cartesian_coords(df.B_x, df.B_y, df.B_z)

In [ ]:
params = ['alpha', 'g', 'orbital', 'V_barrier', 'mu', 'theta', 'phi']
kdims = ['V_bias', 'B']
ds = hv.Dataset(df.set_index(params + kdims).to_xarray())

In [ ]:
%%opts Layout [hspace=0.7]
%%output size=200
(ds.to(hv.Image, kdims=kdims, vdims=['G_01'], groupby=params, dynamic=True) +
 ds.to(hv.Image, kdims=kdims, vdims=['G_Andreev'], groupby=params, dynamic=True)).cols(1)